**Глава 6 (Вежба 1) - Паралелизација во TensorFlow со GPUs**.

Во вежбата се разгледува класификација на слики со конволуциска невронска мрежа со помош на FASHION MNIST податочното множество. Прикажано е како може моделот да биде трениран со паралелизација на еден или повеќе GPUs. 

Примерот е преземен од https://www.tensorflow.org/tutorials/distribute/keras и е соодветно прилагоден.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
datasets, info = tfds.load(name='fashion_mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

In [ ]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
print('Број на уреди: {}'.format(strategy.num_replicas_in_sync))

Број на уреди: 1


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)/255
  return image, label

In [ ]:
train_dataset = mnist_train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.batch(BATCH_SIZE)

In [ ]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
938/938 [==============================] - 15s 12ms/step - loss: 1.5553 - accuracy: 0.5896
Epoch 2/10
938/938 [==============================] - 7s 7ms/step - loss: 0.7244 - accuracy: 0.7068
Epoch 3/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5548 - accuracy: 0.7863
Epoch 4/10
938/938 [==============================] - 7s 7ms/step - loss: 0.4644 - accuracy: 0.8193
Epoch 5/10
938/938 [==============================] - 7s 7ms/step - loss: 0.4186 - accuracy: 0.8331
Epoch 6/10
938/938 [==============================] - 7s 7ms/step - loss: 0.3874 - accuracy: 0.8466
Epoch 7/10
938/938 [==============================] - 7s 7ms/step - loss: 0.3658 - accuracy: 0.8583
Epoch 8/10
938/938 [==============================] - 7s 7ms/step - loss: 0.3465 - accuracy: 0.8655
Epoch 9/10
938/938 [==============================] - 7s 7ms/step - loss: 0.3256 - accuracy: 0.8763
Epoch 10/10
938/938 [==============================] - 7s 7ms/step - loss: 0.3091 - accuracy: 0.88

In [ ]:
eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Евалуациска загуба: {}, Евалуациска точност: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 3s 12ms/step - loss: 0.3255 - accuracy: 0.8859
Евалуациска загуба: 0.3254879415035248, Евалуациска точност: 0.8859000205993652
